# Homework 2. Popularity News RecSys : CS6365 Fall 2025

Justin Mittereder - G49843234

In [123]:
import pandas as pd
import datetime

col_names  = ['Impression ID', 'User ID', 'Time', 'History', 'Impressions']
train_behaviors_df = pd.read_csv('data/MINDsmall_train/behaviors.tsv', sep='\t', header=None, names=col_names)
train_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
train_behaviors_df.reset_index(drop=True, inplace=True)
val_behaviors_df = pd.read_csv('data/MINDsmall_dev/behaviors.tsv', sep='\t', header=None, names=col_names)
val_behaviors_df.sort_values(by=['User ID', 'Time'], inplace=True)
val_behaviors_df.reset_index(drop=True, inplace=True)

col_names  = ['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities']
train_news_df = pd.read_csv('data/MINDsmall_train/news.tsv', sep='\t', header=None, names=col_names)
val_news_df = pd.read_csv('data/MINDsmall_dev/news.tsv', sep='\t', header=None, names=col_names)


In [124]:
train_behaviors_df.head()

,Impression ID,User ID,Time,History,Impressions
0,107691,U100,11/12/2019 7:34:12 AM,N20121 N33998 N45954 N55743 N50095 N18870 N534...,N61235-0 N54489-0 N42597-0 N7800-1 N61408-0 N2...
1,86767,U1000,11/13/2019 11:16:18 PM,N29641 N1789 N41244,N35273-0 N7618-0 N55281-0 N4021-0 N16148-0 N38...
2,74345,U1000,11/14/2019 10:37:21 PM,N29641 N1789 N41244,N14478-0 N58264-0 N7494-0 N46917-0 N22257-0 N1...
3,46640,U1000,11/14/2019 1:00:56 AM,N29641 N1789 N41244,N16148-0 N43628-0 N13094-0 N32774-0 N57034-0 N...
4,155923,U10001,11/11/2019 5:30:21 AM,N47937 N51706 N56447 N61319 N27644 N18030 N272...,N35729-0 N56598-0 N48759-0 N49685-0 N33632-0 N...


In [125]:
print("Len Behaviors: " , len(train_behaviors_df))
print("Unique Users: ", train_behaviors_df['User ID'].nunique())

Len Behaviors:  156965
Unique Users:  50000


In [126]:
train_news_df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [127]:
train_behaviors_df["unix_timestamp"] = pd.to_datetime(
    train_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

val_behaviors_df["unix_timestamp"] = pd.to_datetime(
    val_behaviors_df["Time"], 
    format="%m/%d/%Y %I:%M:%S %p"
).astype("int64") // 1_000_000_000

earliest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].min())
latest_train_time = datetime.datetime.fromtimestamp(train_behaviors_df['unix_timestamp'].max())
earliest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].min())
latest_val_time = datetime.datetime.fromtimestamp(val_behaviors_df['unix_timestamp'].max())

print(f"Training impressions are from between {earliest_train_time} and {latest_train_time} . ")
print(f"Validation impressions are from between {earliest_val_time} and {latest_val_time} . ")


Training impressions are from between 2019-11-08 19:00:19 and 2019-11-14 18:59:13 . 
Validation impressions are from between 2019-11-14 19:00:01 and 2019-11-15 18:58:03 . 


In [128]:
#get full df of all user impressions
import datetime
import pandas as pd

def get_impressions(df, include_history=False):
    rows = []
    for idx, row in df.iterrows():
        user_id = row["User ID"]
        time = row["unix_timestamp"]
        impressions = row["Impressions"]

        # split impressions
        for impression in impressions.split():
            article, click = impression.split("-")
            click = int(click)

            rows.append({
                "user_id": user_id,
                "timestamp": time,
                "article": article,
                "impression": click
            })

        if(include_history):
            # split impressions
            history = str(row['History'])
            for article in history.split(" "):
                rows.append({
                    "user_id": user_id,
                    "timestamp": "N/A",
                    "article": article,
                    "impression": 1 
                })
        
    impressions_df = pd.DataFrame(rows)
    impressions_df.sort_values(by=['user_id', 'timestamp'], inplace=True)
    impressions_df.reset_index(drop=True, inplace=True)
    return impressions_df    


impressions_df = get_impressions(train_behaviors_df)
print("Number of Impressions: ", len(impressions_df))
impressions_df.head()


Number of Impressions:  5843444


,user_id,timestamp,article,impression
0,U100,1573544052,N61235,0
1,U100,1573544052,N54489,0
2,U100,1573544052,N42597,0
3,U100,1573544052,N7800,1
4,U100,1573544052,N61408,0


Get the most popular articles based on the number of successful impressions (clicks) in the current timestamp

In [129]:
success_impressions_df = impressions_df[impressions_df['impression'] == 1]
print("num successful impressions: ", len(success_impressions_df))
print("unique articles that have been clicked on: ", success_impressions_df['article'].nunique())

# Count clicks per user
click_counts_user = (success_impressions_df.groupby("user_id").size().reset_index(name="click_count"))
click_counts_user.sort_values('click_count', ascending=False, inplace=True)
print("Impressions User: \n", click_counts_user)
users_with_1_click = click_counts_user[click_counts_user['click_count'] == 1]
print("Size users with 1 impression: ", len(users_with_1_click))
print("Unique Users with impressions : " , click_counts_user['user_id'].nunique())
print("Unique Users with 1 impression : ", users_with_1_click['user_id'].nunique())

# Count impressions per article
impression_counts = (success_impressions_df.groupby("article").size().reset_index(name="impression_count"))
impression_counts.sort_values('impression_count', ascending=False, inplace=True)
#print("Impressions Article: \n", impression_counts)

# Sort by most clicked
popular_items = impression_counts.sort_values("impression_count", ascending=False)
print(popular_items)

# Convert to top 1000 list (most clicked first)
pop_list = popular_items["article"].head(1000).tolist()
print(pop_list)

num successful impressions:  236344
unique articles that have been clicked on:  7713
Impressions User: 
       user_id  click_count
25379  U53220          129
35630  U70550          120
31460  U63482          112
6454   U20833           98
13226  U32322           96
...       ...          ...
26392  U54966            1
26375  U54934            1
26370  U54927            1
26358  U54909            1
25000  U52589            1

[50000 rows x 2 columns]
Size users with 1 impression:  12810
Unique Users with impressions :  50000
Unique Users with 1 impression :  12810
     article  impression_count
5951  N55689              4316
3318  N35729              3346
3028  N33619              3246
5680  N53585              2835
7076  N63970              2578
...      ...               ...
7258   N6642                 1
381   N12977                 1
546   N14323                 1
545   N14320                 1
0     N10032                 1

[7713 rows x 2 columns]
['N55689', 'N35729', 'N33619', '

In [130]:
print(val_behaviors_df['User ID'].nunique())
print(len(val_behaviors_df))
val_counts = val_behaviors_df['User ID'].value_counts()
print(val_counts)
val_more_than_1 = val_counts > 1
len(val_behaviors_df)
print(val_more_than_1[val_more_than_1 == True].sum())
print()

50000
73152
User ID
U38741    18
U67841    15
U86264    12
U82600    12
U32122    12
          ..
U52723     1
U52722     1
U52719     1
U52716     1
U1         1
Name: count, Length: 50000, dtype: int64
14826



In [ ]:
val_behaviors_df.head()
print(val_behaviors_df.iloc[0]['History'])
print(type(val_behaviors_df.iloc[0]['History']))
print("Empty Histories: " , val_behaviors_df["History"].isna().sum())
print("Not Empty Histories: ", val_behaviors_df['History'].notna().sum())



N23571 N58267 N25682 N10646 N32607 N57737 N52301 N596 N24356 N13374 N40207 N62058
<class 'str'>
Empty Histories:  2214
Not Empty Histories:  70938
  user_id                                            article
0      U1                                           [N20036]
1     U10                                           [N32536]
2  U10000                           [N31958, N50775, N60215]
3  U10002  [N50775, N436, N64264, N56434, N35676, N5940, ...
4  U10004                                   [N36779, N33176]


In [133]:
#aggregate the histories for each unique user
#since there are rows with the same User ID, we need to add the histories for each user together before going to the get_popularity_recs function
val_behaviors_df["History_list"] = val_behaviors_df["History"].str.split()
# Aggregate lists per user
val_behaviors_df["History_list"] = val_behaviors_df["History_list"].apply(lambda x: x if isinstance(x, list) else []) # make all empty Histories an empty list
val_df_grouped = val_behaviors_df.groupby("User ID")["History_list"].agg(lambda lists: sum(lists, [])).reset_index()
val_df_grouped["History_list"] = val_df_grouped["History_list"].apply(lambda x: list(set(x)))
print(val_df_grouped.head())

  User ID                                       History_list
0      U1  [N58267, N13374, N10646, N62058, N24356, N3260...
1     U10  [N57967, N64777, N9120, N9803, N36699, N2945, ...
2  U10000  [N40716, N46978, N19434, N47173, N2231, N63709...
3  U10002  [N29841, N1998, N37942, N25532, N50, N39117, N...
4  U10004  [N33859, N27251, N43482, N38118, N15627, N1887...


In [137]:
#For each user in val and test sets, get their recommendations while also filtering out their previously viewed articles in the 'History' field
def get_popularity_recs(df, k):
    user_recs = {}
    for _, row in df.iterrows():
        user_id = row["User ID"]
        #gets users previously clicked articles
        history = row['History_list']
        # for each item in the popularity list, add it to recommendations if its not in the users history
        filtered = [item for item in pop_list if item not in history]

        #grab top k
        user_recs[user_id] = filtered[:k]
    return user_recs

val_user_recs_10 = get_popularity_recs(val_df_grouped, k=10)
val_user_recs_50 = get_popularity_recs(val_df_grouped, k=50)


In [138]:
#get all clicked articles into a df to check for hit rate against this list of ground truth
val_impressions = get_impressions(val_behaviors_df)
val_impressions_clicked = val_impressions[val_impressions["impression"] == 1]
val_impressions_grouped = val_impressions_clicked.groupby("user_id")["article"].apply(list).reset_index()
print(val_impressions_grouped.head())

  user_id                                            article
0      U1                                           [N20036]
1     U10                                           [N32536]
2  U10000                           [N31958, N50775, N60215]
3  U10002  [N50775, N436, N64264, N56434, N35676, N5940, ...
4  U10004                                   [N36779, N33176]


In [139]:
# checks if the actual article clicked in the current validation impression is included in recs for each user 
# compares number of hits to the total number of articles clicked to get the hit rate with k items being recommended
def hit_rate_at_k(df,recs):
    hits = 0
    for _, row in df.iterrows():
        user_id = row["user_id"]
        articles = row["article"]
        for article in articles:
            if user_id in recs and article in recs[user_id]: #if we recommended one article that the user actually clicked on
                #print("got a hit")
                hits += 1
                break #if we get a hit, only count one hit per user or no hits per user 

    return (hits / len(df))*100

val_hr10 = hit_rate_at_k(val_impressions_grouped, val_user_recs_10)
val_hr50 = hit_rate_at_k(val_impressions_grouped, val_user_recs_50)
print(f"Validation Hit Rate@10: {val_hr10}")
print(f"Validation Hit Rate@50: {val_hr50}")

Validation Hit Rate@10: 0.122
Validation Hit Rate@50: 1.0619999999999998
